In [ ]:
# Explain stack semantics in Trax
# Select and Residual operate on elements in the stack

import numpy as np
from trax import layers as tl
from trax import shapes
from trax import fastmath

In [ ]:
# stack example

def Addition():
    layer_name = "Addition"
    
    def func(x, y):
        return x + y
    
    return tl.Fn(layer_name, func)

add = Addition()

print(add.name)
print(add.n_in)
print(add.n_out)

x = np.array([3])
y = np.array([4])
print(x)
print(y)
z = add((x, y))
print(z)

In [ ]:
def Multiplication():
    layer_name = "Multiplication"
    
    def func(x, y):
        return x * y
    
    return tl.Fn(layer_name, func)

mul = Multiplication()

print(mul.name)
print(mul.n_in)
print(mul.n_out)

x = np.array([7])
y = np.array([15])
print(x)
print(y)
z = mul((x, y))
print(z)

In [ ]:
serial = tl.Serial(
    Addition(), Multiplication(), Addition() # (3 + 4) * 15 + 3
)
x = (np.array([3]), np.array([4]), np.array([15]), np.array([3]))

serial.init(shapes.signature(x))

print("Serial Model")
print(serial)
print(serial.name)
print(serial.sublayers)
print(serial.n_in)
print(serial.n_out)
print(x)
print(serial(x))

In [ ]:
# tl.Select combinator

serial = tl.Serial(tl.Select([0,1, 0, 1]), Addition(), Multiplication(), Addition())
x = (np.array([3]), np.array([4]))

serial.init(shapes.signature(x))

print(serial)
print(serial.name)
print(serial.sublayers)
print(serial.n_in)
print(serial.n_out)

print(x)

print(serial(x))

In [ ]:
serial = tl.Serial(
    tl.Select([0, 1, 0, 1]), Addition(), tl.Select([0], n_in=2), Multiplication()
)

x = (np.array([3]), np.array([4]))
serial.init(shapes.signature(x))
print(x)
print(serial(x))

# Select copies the inputs in  order to be used further along in the  stack of operations

In [ ]:
# tl.Residual combinator. Residual networks make deep models easier to train.
# Residual computes the element-wise sum of the stack-top input with the output
# of the layer series

serial = tl.Serial(
    tl.Select([0, 1, 0, 1]),
    tl.Residual(Addition())
)

print(serial)

x1 = np.array([3])
x2 = np.array([4])

print((x1, x2))
print(serial((x1, x2)))

# BLEU Scores
- Closer to 1 is better

In [ ]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
import numpy as np
import nltk
from nltk.util import ngrams
nltk.download('punkt')
import math
from collections import Counter
import sacrebleu
import matplotlib.pyplot as plt

In [ ]:
# Brevity Penalty
ref_length= np.ones(100)
can_length = np.linspace(1.5, 0.5, 100)
x = ref_length / can_length
y =  1 - x
y = np.exp(y)
y = np.minimum(np.ones(y.shape), y)

fig, ax = plt.subplots(1)
lines = ax.plot(x, y)
ax.set(
    xlabel='ratio of the length of the reference to the candidate text',
    ylabel='Brevity Penalty'
)
plt.show()

In [ ]:
data = {"1-gram": 0.8, "2-gram": 0.7, "3-gram": 0.6, "4-gram": 0.5}
names = list(data.keys())
values = list(data.values())

fig, ax = plt.subplots(1)
bars = ax.bar(names, values)
ax.set(ylabel="N-gram precision")

plt.show()

In [ ]:
data = {"1-gram": 0.8, "2-gram": 0.77, "3-gram": 0.74, "4-gram": 0.71}
names = list(data.keys())
values = list(data.values())

fig, ax = plt.subplots(1)
bars = ax.bar(names, values)
ax.set(ylabel="Modified N-gram precision")

plt.show()

# When the n-gram precision is multiplied by the BP, then the exponential
# decay of n-grams is almost fully compensated. The BLEU score corresponds
# to a geometric average of this modified n-gram precision

In [ ]:
# Example calculation
reference = "The NASA Opportunity rover is battling a massive dust storm on planet Mars."
candidate_1 = "The Opportunity rover is combating a big sandstorm on planet Mars."
candidate_2 = "A NASA rover is fighting a massive storm on planet Mars."

tokenized_ref = nltk.word_tokenize(reference.lower())
tokenized_cand_1 = nltk.word_tokenize(candidate_1.lower())
tokenized_cand_2 = nltk.word_tokenize(candidate_2.lower())

print(f"{reference} -> {tokenized_ref}")
print("\n")
print(f"{candidate_1} -> {tokenized_cand_1}")
print("\n")
print(f"{candidate_2} -> {tokenized_cand_2}")

In [ ]:
def brevity_penalty(candidate, reference):
    ref_length = len(reference)
    can_length = len(candidate)
    
    if ref_length < can_length:
        BP = 1
    else:
        penalty = 1 - (ref_length / can_length)
        BP = np.exp(penalty)
    
    return BP

In [ ]:
def clipped_precision(candidate, reference):
    """
    Clipped precision function given a original and a machine translated sentences
    """

    clipped_precision_score = []
    
    for i in range(1, 5):
        ref_n_gram = Counter(ngrams(reference,i))
        cand_n_gram = Counter(ngrams(candidate,i))

        c = sum(cand_n_gram.values())
        
        for j in cand_n_gram: # for every n-gram up to 4 in candidate text
            if j in ref_n_gram: # check if it is in the reference n-gram
                if cand_n_gram[j] > ref_n_gram[j]: # if the count of the candidate n-gram is bigger
                                                   # than the corresponding count in the reference n-gram,
                    cand_n_gram[j] = ref_n_gram[j] # then set the count of the candidate n-gram to be equal
                                                   # to the reference n-gram
            else:
                cand_n_gram[j] = 0 # else set the candidate n-gram equal to zero

        clipped_precision_score.append(sum(cand_n_gram.values())/c)

    weights =[0.25]*4

    s = (w_i * math.log(p_i) for w_i, p_i in zip(weights, clipped_precision_score))
    s = math.exp(math.fsum(s))
    return s



In [ ]:
def bleu_score(candidate, reference):
    BP = brevity_penalty(candidate, reference)
    precision = clipped_precision(candidate, reference)
    return BP * precision

print(
    "Results reference versus candidate 1 our own code BLEU: ",
    round(bleu_score(tokenized_cand_1, tokenized_ref) * 100, 1),
)
print(
    "Results reference versus candidate 2 our own code BLEU: ",
    round(bleu_score(tokenized_cand_2, tokenized_ref) * 100, 1),
)

In [ ]:
print(
    "Results reference versus candidate 1 sacrebleu library BLEU: ",
    round(sacrebleu.corpus_bleu(candidate_1, reference).score, 1),
)
print(
    "Results reference versus candidate 2 sacrebleu library BLEU: ",
    round(sacrebleu.corpus_bleu(candidate_2, reference).score, 1),
)

In [ ]:
# Loading the raw data
wmt19_src = open("wmt19_src.txt", "rU")
wmt19_src_1 = wmt19_src.read()
wmt19_src.close()
wmt19_ref = open("wmt19_ref.txt", "rU")
wmt19_ref_1 = wmt19_ref.read()
wmt19_ref.close()
wmt19_can = open("wmt19_can.txt", "rU")
wmt19_can_1 = wmt19_can.read()
wmt19_can.close()

tokenized_corpus_src = nltk.word_tokenize(wmt19_src_1.lower())
tokenized_corpus_ref = nltk.word_tokenize(wmt19_ref_1.lower())
tokenized_corpus_cand = nltk.word_tokenize(wmt19_can_1.lower())

In [ ]:
print("English source text:")
print("\n")
print(f"{wmt19_src_1[0:170]} -> {tokenized_corpus_src[0:30]}")
print("\n")
print("German reference translation:")
print("\n")
print(f"{wmt19_ref_1[0:219]} -> {tokenized_corpus_ref[0:35]}")
print("\n")
print("German machine translation:")
print("\n")
print(f"{wmt19_can_1[0:199]} -> {tokenized_corpus_cand[0:29]}")

In [ ]:
print(
    "Results reference versus candidate 1 our own BLEU implementation: ",
    round(bleu_score(tokenized_corpus_cand, tokenized_corpus_ref) * 100, 1),
)

In [ ]:
print(
    "Results reference versus candidate 1 sacrebleu library BLEU: ",
    round(sacrebleu.corpus_bleu(wmt19_can_1, wmt19_ref_1).score, 1),
)

In [ ]:
# BLEU score >40 is high quality! >60 is often better than human.